In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import pandas as pd

In [2]:
# Configurações do Edge para rodar em modo headless
options = Options()
options.add_argument("--headless")

# Instalação e configuração do serviço Edge
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(options=options, service=service)


In [36]:
def get_data(url, driver):
    print("Extraindo dados de: " + url)
    # Extrair o ano da URL
    year = url.split("serie-a-")[-1].split("/results")[0]  # Extrai o ano diretamente antes de "/results"

    driver.get(url)
    sleep(10)
    
    # Rolagem para o final da página para garantir que todos os elementos sejam carregados
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(10)  # Espera adicional para garantir que a página tenha carregado completamente após a rolagem

    gameRows = driver.find_elements(By.XPATH, '//div[@class="group flex"]')
    rowData = []

    # Adiciona o ano como primeira coluna
    for row in gameRows:
        row_content = row.text.split("\n")
        rowData.append([year] + row_content)

    return rowData

In [43]:
def get_all_data(years, pages):
    base_url = "https://www.oddsportal.com/football/brazil/serie-a-{year}/results/#/page/{page}/"
    all_data = []

    for year in years:
        for page in range(1, pages + 1):
           driver = webdriver.Edge(options=options, service=service)
           url = base_url.format(year=year, page=page)
           data = get_data(url, driver)
           all_data.extend(data)
           driver.quit()  # Fechar o navegador após terminar a extração

    return all_data

# Anos a serem coletados
years = [2021, 2022, 2023]
# Número de páginas por ano
pages = 8

# Coleta os dados
collected_data = get_all_data(years, pages)


Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/1/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/2/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/3/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/4/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/5/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/6/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/7/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2021/results/#/page/8/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2022/results/#/page/1/
Extraindo dados de: https://www.oddsportal.com/football/brazil/serie-a-2022/results/#/page/2/
Extraindo dados de: https://www.oddsportal.com/football/braz

In [52]:
# Criar um DataFrame a partir da lista
df = pd.DataFrame(collected_data, columns=['Year', 'Time', 'Home Team', 'Home Score', 'vs', 'Away Score', 'Away Team', 'Odd 1', 'Odd X', 'Odd 2', 'Bookmakers'])

# Limpar colunas desnecessárias
df.drop(columns=['vs', 'Bookmakers'], inplace=True)

# Salvar o DataFrame em um arquivo csv
df.to_csv('brasileirao_odds.csv', index=False)
# Salvar o DataFrame em um arquivo excel
df.to_excel('brasileirao_odds.xlsx', index=False)

print(df)

      Year   Time    Home Team Home Score Away Score       Away Team Odd 1  \
0     2021  21:30   America MG          2          0       Sao Paulo  +100   
1     2021  21:30  Atletico GO          2          0     Flamengo RJ  -139   
2     2021  21:30   Bragantino          1          0   Internacional  -111   
3     2021  21:30    Juventude          1          0     Corinthians  +125   
4     2021  21:30   Fluminense          3          0  Chapecoense-SC  -526   
...    ...    ...          ...        ...        ...             ...   ...   
1135  2023  18:30  Botafogo RJ          2          1       Sao Paulo  +141   
1136  2023  18:30   Bragantino          2          1           Bahia  -118   
1137  2023  18:30    Fortaleza          1          1   Internacional  -109   
1138  2023  16:00   America MG          0          3      Fluminense  +152   
1139  2023  16:00    Palmeiras          2          1          Cuiaba  -278   

     Odd X  Odd 2  
0     +231   +298  
1     +289   +357  
2  

In [53]:
# contando o numero de dados para year = 2021
print(df[df['Year'] == '2021'].count())

Year          380
Time          380
Home Team     380
Home Score    380
Away Score    380
Away Team     380
Odd 1         380
Odd X         380
Odd 2         380
dtype: int64


In [54]:
# verfica se tem algum valor nulo
print(df.isnull().sum())

Year          0
Time          0
Home Team     0
Home Score    0
Away Score    0
Away Team     0
Odd 1         0
Odd X         0
Odd 2         0
dtype: int64


In [55]:
# verifica se tem algum valor duplicado
print(df.duplicated().sum())

0
